# Set Estimator Options

## Introduction

This notebook demonstrates how to configure the **Estimator V2** primitive options, including error suppression (Twirling) and error mitigation (Resilience). We will run actual jobs to visualize the `EstimatorResult` output.

In [ ]:
# Setup: Define Backend and Circuit
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# 1. Initialize Backend
backend = FakeManilaV2()

# 2. Create Circuit and Observables
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

observable = SparsePauliOp(["ZZ", "IZ", "ZI"], coeffs=[1.0, 0.5, 0.5])

# 3. Transpile to ISA
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_qc = pm.run(qc)
isa_observable = observable.apply_layout(isa_qc.layout)

# 4. Define Pub
pub = (isa_qc, isa_observable)

print("Setup Complete.")

## 1. Default Execution & Result Format

First, let's run the Estimator with default options to understand the output format.

In [ ]:
estimator = Estimator(mode=backend)

# Run Job
job = estimator.run([pub])
print(f"Job ID: {job.job_id()}")

# Get Result
result = job.result()

# Analyze Result
print(f"\n>>> Full Result Object: {result}")
# The 'evs' field contains the array of expectation values
print(f"\n>>> Expectation Values (evs): {result[0].data.evs}")
print(f">>> Metadata: {result[0].metadata}")

## 2. Twirling Options (Error Suppression)

Pauli Twirling randomizes the circuit during execution to convert coherent errors into stochastic noise. This is often enabled by default in resilience levels > 0, but can be explicitly controlled.

In [ ]:
estimator_twirl = Estimator(mode=backend)

# Configure Twirling
estimator_twirl.options.twirling.enable_gates = True
estimator_twirl.options.twirling.num_randomizations = 32
estimator_twirl.options.twirling.shots_per_randomization = 100

# Run with Twirling
job_twirl = estimator_twirl.run([pub])
result_twirl = job_twirl.result()

# Notice we again access .data.evs to get the result of the twirled execution
print(f"Twirled Expectation Values (via .data.evs): {result_twirl[0].data.evs}")

## 3. Resilience Options (Error Mitigation)

Qiskit Runtime supports various resilience methods like ZNE (Zero Noise Extrapolation) and PEC (Probabilistic Error Cancellation). 

> **Note:** Advanced mitigation strategies like PEC and ZNE typically require the full Qiskit Runtime service environment. The local `FakeManilaV2` may not perform the full extrapolation math but accepts the API options.

In [ ]:
estimator_zne = Estimator(mode=backend)

# Configure ZNE
estimator_zne.options.resilience.zne_mitigation = True
estimator_zne.options.resilience.zne.noise_factors = (1, 3, 5)
estimator_zne.options.resilience.zne.extrapolator = "exponential"

# Run with ZNE options
job_zne = estimator_zne.run([pub])
result_zne = job_zne.result()

# The Result object remains the same structure, even with ZNE applied.
print(f"ZNE Configured - Expectation Values (via .data.evs): {result_zne[0].data.evs}")

## Summary

The `EstimatorResult` provides:
1.  **Values (`evs`)**: The calculated expectation values.
2.  **Standard Errors (`stds`)**: Statistical uncertainty (if shots > 0).
3.  **Metadata**: Settings used for the run, including precision and shots.

By accessing `result[i].data.evs`, you retrieve the core physics output of the primitive.